In [21]:
from heapq import heappush, heappop

class Node:
  def __init__(self, state, parent, depth):
    self.state = state
    self.parent = parent
    self.depth = depth

  def __lt__(self, other):
    return self.state < other.state

class PriorityQueue:
  def __init__(self):
    self.list = []

  def push(self, value, content):
    """ Add a node into the queue with priority value """
    # The middle value in the tuple is used as a tiebreaker
    # if the priority value is the same for multiple nodes.
    heappush(self.list, (value, len(self.list), content))

  def pop(self):
    """ Get the node with the smallest priority value """
    return heappop(self.list)[2]

  def __len__(self):
    """ Get the size of the queue. You can call len(queue) """
    return len(self.list)

class Searcher:
  def __init__(self):
    self.expanded_nodes_count = 0

  def search(self, start):
    pass

class State:
  def __init__(self, grid):
    self.grid = grid

  def __hash__(self):
    return hash((self.grid[0][0], self.grid[0][1], self.grid[0][2], \
                 self.grid[1][0], self.grid[1][1], self.grid[1][2], \
                 self.grid[2][0], self.grid[2][1], self.grid[2][2], ))

  def __eq__(self, other):
    return self.grid == other.grid



In [22]:
def successors(state):
    # TODO implement
    successors = []
    row = None
    col = None

    #Find the 0
    for i in range(3):
        for j in range(3):
          if state[i][j] == 0:
            row = i
            col = j
            break
        if row is not None:
          break

    next_dir = [(1,0), (0,1), (-1,0), (0,-1)]

    for r, c in next_dir:
      new_row = row + r
      new_col = col + c
      if new_row >= 0 and new_row < 3 and new_col >= 0 and new_col < 3:
        new_grid = [row[:] for row in state]
        new_grid[row][col] = new_grid[new_row][new_col]
        new_grid[new_row][new_col] = 0

        successors.append(new_grid)
    return successors

GOAL = [[1,2,3],[4,5,6],[7,8,0]]

def is_goal(node):
    # TODO implement
    return node.state.grid == GOAL

In [23]:
# Example test cases
print(successors([[1,3,6],[4,0,5],[7,8,2]]))
#  [[[1, 3, 6], [4, 8, 5], [7, 0, 2]], \
#  [[1, 3, 6], [4, 5, 0], [7, 8, 2]], \
#  [[1, 0, 6], [4, 3, 5], [7, 8, 2]], \
#  [[1, 3, 6], [0, 4, 5], [7, 8, 2]]]

print(successors([[7,1,2],[5,6,3],[0,4,8]]))
# [[[7, 1, 2], [5, 6, 3], [4, 0, 8]], [[7, 1, 2], [0, 6, 3], [5, 4, 8]]]

[[[1, 3, 6], [4, 8, 5], [7, 0, 2]], [[1, 3, 6], [4, 5, 0], [7, 8, 2]], [[1, 0, 6], [4, 3, 5], [7, 8, 2]], [[1, 3, 6], [0, 4, 5], [7, 8, 2]]]
[[[7, 1, 2], [5, 6, 3], [4, 0, 8]], [[7, 1, 2], [0, 6, 3], [5, 4, 8]]]


In [24]:
from collections import deque

class BfsSearcher(Searcher):
  def __init__(self):
    super().__init__()
    
  def search(self, start):
    # TODO implement
    start_state = State(start)
    queue = deque([Node(start_state, None, 0)])

    visited = set()
    visited.add(start_state)

    while queue:
      curr = queue.popleft()
      self.expanded_nodes_count += 1

      if is_goal(curr):
        return curr
      
      for successor_grid in successors(curr.state.grid):
        successor_state = State(successor_grid)
        if successor_state not in visited:
          visited.add(successor_state)
          queue.append(Node(successor_state, curr, curr.depth + 1))

    return None

In [25]:
import time

start_easy = [[1,2,3],[5,0,7],[4,8,6]]

start_hard = [[1,2,3],[4,7,6],[0,8,5]]


for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")
  t = time.time()
  searcher = BfsSearcher()
  sol = searcher.search(state)
  elapsed_time = time.time() - t
  print("Elapsed time for BFS:", elapsed_time)
  print("Expanded nodes for BFS:", searcher.expanded_nodes_count)
  print("Depth for BFS:", sol.depth)

=== State 0 ===
Elapsed time for BFS: 0.001007080078125
Expanded nodes for BFS: 244
Depth for BFS: 8
=== State 1 ===
Elapsed time for BFS: 0.05431628227233887
Expanded nodes for BFS: 7573
Depth for BFS: 16


In [26]:
class IdsSearcher(Searcher):
  def __init__(self):
    super().__init__()
  def search(self, start):
    # TODO implement
    start_node = Node(State(start),None,0)
    depth_limit = 0

    while True:
      visited_depth = {}

      stack = [start_node]
      goal_state = None

      while stack:
        curr = stack.pop()

        if is_goal(curr):
          goal_state = curr
          break
        
        if curr.depth == depth_limit:
          continue

        previous_depth = visited_depth.get(curr.state, float('inf'))
        if curr.depth >= previous_depth:
          continue

        visited_depth[curr.state] = curr.depth
        self.expanded_nodes_count += 1

        for successor_grid in successors(curr.state.grid):
          child_state = State(successor_grid)
          child_node = Node(child_state,parent=curr,depth=curr.depth+1)
          stack.append(child_node)
      
      if goal_state is not None:
        return goal_state
      
      depth_limit += 1




In [27]:
# Run the algorithm

for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")
  t = time.time()
  searcher = IdsSearcher()
  sol = searcher.search(state)
  elapsed_time = time.time() - t
  print("Elapsed time for IDS:", elapsed_time)
  print("Expanded nodes for IDS:", searcher.expanded_nodes_count)
  print("Depth for IDS:", sol.depth)

=== State 0 ===
Elapsed time for IDS: 0.0010180473327636719
Expanded nodes for IDS: 421
Depth for IDS: 8
=== State 1 ===
Elapsed time for IDS: 0.08628678321838379
Expanded nodes for IDS: 14614
Depth for IDS: 16


In [28]:
def manhattan_heuristic(node):
    # TODO implement
   if isinstance(node,Node):
      node = node.state.grid
   distance_sum = 0
   for i in range(3):
      for j in range(3):
         val = node[i][j]
         if val != 0:
            goal_i = (val - 1) // 3
            goal_j = (val - 1) % 3

            distance_sum += abs(i - goal_i) + abs(j - goal_j)
   return distance_sum

def misplaced_tiles_heuristic(node):
    # TODO implement
    if isinstance(node,Node):
       node = node.state.grid
    m = 0
    for i in range(3):
       for j in range(3):
          val = node[i][j]
          if val != 0 and not val == GOAL[i][j]:
             m += 1
    return m

class AStarSearcher(Searcher):
    def __init__(self):
        super().__init__()
    def search(self, start, heuristic):
        # TODO implement
        c = 0
        start_state = State(start)
        start_node = Node(start_state, None, 0)

        frontier = PriorityQueue()
        frontier.push((heuristic(start_node), c), start_node)
        c += 1

        cost_so_far = {start_node.state: 0}

        while len(frontier) > 0:
            curr = frontier.pop()
            
            if is_goal(curr):
                return curr

            self.expanded_nodes_count += 1

            current_cost = cost_so_far[curr.state]

            for successor_grid in successors(curr.state.grid):
                successor_state = State(successor_grid)
                new_cost = current_cost + 1

                if successor_state not in cost_so_far or new_cost < cost_so_far[successor_state]:
                    cost_so_far[successor_state] = new_cost
                    succ_node = Node(successor_state, curr, curr.depth + 1)
                    priority = new_cost + heuristic(succ_node)
                    
                    frontier.push((priority, c), succ_node)
                    c += 1

        return None


In [31]:
# Example test cases
print(manhattan_heuristic([[7,1,2],[5,6,3],[0,4,8]]))
# should be 10

print(misplaced_tiles_heuristic([[1,2,3],[0,5,6],[4,7,8]]))
# should be 3

10
3


In [33]:
for i, state in enumerate([start_easy, start_hard]):
  print(f"=== State {i} ===")

  t = time.time()
  searcher = AStarSearcher()
  sol = searcher.search(state, manhattan_heuristic)
  elapsed_time = time.time() - t
  print("Elapsed time for A star with Manhattan:", elapsed_time)
  print("Expanded nodes for A star", searcher.expanded_nodes_count)
  print("Depth for BFS:", sol.depth)

  t = time.time()
  searcher = AStarSearcher()
  sol = searcher.search(state, misplaced_tiles_heuristic)
  elapsed_time = time.time() - t
  print("Elapsed time for A star with Misplaced Tiles:", elapsed_time)
  print("Expanded nodes for A star", searcher.expanded_nodes_count)
  print("Depth for BFS:", sol.depth)


=== State 0 ===
Elapsed time for A star with Manhattan: 0.0010056495666503906
Expanded nodes for A star 15
Depth for BFS: 8
Elapsed time for A star with Misplaced Tiles: 0.0
Expanded nodes for A star 26
Depth for BFS: 8
=== State 1 ===
Elapsed time for A star with Manhattan: 0.0019981861114501953
Expanded nodes for A star 290
Depth for BFS: 16
Elapsed time for A star with Misplaced Tiles: 0.006000518798828125
Expanded nodes for A star 669
Depth for BFS: 16


The number of nodes expanded by BFS and IDS grows exponentially with the depth of the solution, quickly making them very expensive for deeper puzzles. On the other hand, A* with an admissible heuristic, particularly Manhattan distance, greatly reduces the number of node expansions. Even though Manhattan distance performs better than the Misplaced Tiles heuristic, both versions of A* still outperform BFS and IDS by a notable margin.